In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from torchvision.models import resnet34
import numpy as np

In [2]:
# CutMix数据增强
def cutmix_data(x, y, alpha=1.0):
    indices = torch.randperm(x.size(0))
    shuffled_x = x[indices]
    shuffled_y = y[indices]
    lam = np.random.beta(alpha, alpha)

    bbx1, bby1, bbx2, bby2 = rand_bbox(x.size(), lam)
    x[:, :, bbx1:bbx2, bby1:bbx2] = shuffled_x[:, :, bbx1:bbx2, bby1:bbx2]

    return x, y, shuffled_y, lam

def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)

    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# 数据预处理
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
])

# 加载数据集
trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# 定义ResNet-34模型
class ResNetCIFAR(nn.Module):
    def __init__(self):
        super(ResNetCIFAR, self).__init__()
        self.model = resnet34(weights=None)
        self.model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.model.maxpool = nn.Identity()
        self.model.fc = nn.Linear(512, 100)

    def forward(self, x):
        return self.model(x)

In [4]:
net = ResNetCIFAR()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = net.to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

# 初始化TensorBoard
writer = SummaryWriter('resnet/experiment_resnet34')

# 记录最高准确率
best_acc = 0

In [5]:
# 训练模型
for epoch in range(100):  # 共训练100个epoch
    net.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # 应用CutMix
        inputs, targets_a, targets_b, lam = cutmix_data(inputs, labels)

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = lam * criterion(outputs, targets_a) + (1 - lam) * criterion(outputs, targets_b)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (lam * (predicted == targets_a).sum().item() + (1 - lam) * (predicted == targets_b).sum().item())

    train_loss = running_loss / len(trainloader)
    train_accuracy = 100 * correct / total

    writer.add_scalar('training loss', train_loss, epoch)
    writer.add_scalar('training accuracy', train_accuracy, epoch)


    scheduler.step()

    # 验证模型
    net.eval()
    correct = 0
    total = 0
    val_loss = 0.0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(testloader)
    val_accuracy = 100 * correct / total

    writer.add_scalar('validation loss', val_loss, epoch)
    writer.add_scalar('validation accuracy', val_accuracy, epoch)

    print(f'Epoch {epoch + 1}: Training Loss: {train_loss:.3f}, Training Accuracy: {train_accuracy:.2f}%, Validation Loss: {val_loss:.3f}, Validation Accuracy: {val_accuracy:.2f}%')

    # 保存验证集上最高准确率的模型
    if val_accuracy > best_acc:
        best_acc = val_accuracy
        torch.save(net.state_dict(), 'resnet34.pth')

writer.close()

/root/miniconda3/lib/python3.12/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 1: Training Loss: 4.363, Training Accuracy: 5.13%, Validation Loss: 3.720, Validation Accuracy: 11.49%
Epoch 2: Training Loss: 4.140, Training Accuracy: 8.12%, Validation Loss: 3.419, Validation Accuracy: 17.54%
Epoch 3: Training Loss: 3.938, Training Accuracy: 11.96%, Validation Loss: 3.066, Validation Accuracy: 25.24%
Epoch 4: Training Loss: 3.751, Training Accuracy: 15.49%, Validation Loss: 2.817, Validation Accuracy: 28.65%
Epoch 5: Training Loss: 3.655, Training Accuracy: 17.46%, Validation Loss: 2.587, Validation Accuracy: 34.50%
Epoch 6: Training Loss: 3.564, Training Accuracy: 20.40%, Validation Loss: 2.477, Validation Accuracy: 37.00%
Epoch 7: Training Loss: 3.396, Training Accuracy: 23.63%, Validation Loss: 2.231, Validation Accuracy: 42.37%
Epoch 8: Training Loss: 3.299, Training Accuracy: 26.07%, Validation Loss: 2.066, Validation Accuracy: 46.77%
Epoch 9: Training Loss: 3.295, Training Accuracy: 26.41%, Validation Loss: 2.087, Validation Accuracy: 48.21%
Epoch 10: Tr